In [1]:
import sqlite3
import plyvel
from pathlib import Path
import re
import json
from functions.runHealth import runHealth
# from functions.dynamic_analysis import Dynamic
import requests

In [2]:
# path : Path = Path('/home/ndanner_plp/OpenWPM/crawl-data/datadir8000')
path : Path = Path('../../datadirTesting2')
con : sqlite3.Connection = sqlite3.connect( str(path.joinpath("crawl-data.sqlite")) )
db = plyvel.DB( str(path.joinpath("leveldb")) )
%load_ext sql
%sql sqlite:///{path}/crawl-data.sqlite

'Connected: @../../datadirTesting2/crawl-data.sqlite'

In [ ]:
n, f = runHealth(con)
print(f"total visits: {n}, failed/incomplete visits: {f}. Success percentage: {round(100* (1 - f/n)) }%")

In [ ]:
for key, value in db:
    print(key)
    # print(value, end="\n\n")

In [ ]:
# results = Dynamic(con)
# print( json.dumps( results, indent= 4 ) )

In [ ]:
# %load_ext sql
# symbols = {}
# for path in Path("/home/ndanner_plp/OpenWPM/crawl-data").iterdir():
#     if path.is_dir() and re.match("^datadir" ,path.name):
#         %sql sqlite:///{path}/crawl-data.sqlite
#         result = %sql SELECT DISTINCT(symbol) FROM javascript
#         if result:
#             for symbol in result.DataFrame()['symbol']:
#                 if symbol in symbols:
#                     symbols[symbol].append(path.name)
#                 else:
#                     symbols[symbol] = [path.name]
# print( json.dumps( symbols, indent= 4 ) )

OfflineAudioContext

In [3]:
%%sql
SELECT *
FROM javascript
LIMIT 10

 * sqlite:///../../datadirTesting2/crawl-data.sqlite
Done.


id,incognito,browser_id,visit_id,extension_session_uuid,event_ordinal,page_scoped_event_ordinal,window_id,tab_id,frame_id,script_url,script_line,script_col,func_name,script_loc_eval,document_url,top_level_url,call_stack,symbol,operation,value,arguments,time_stamp


In [ ]:
%%sql
WITH a AS (
SELECT visit_id,url, COUNT(*) as n
FROM http_responses
WHERE content_hash <> ""
GROUP BY visit_id,url
ORDER BY n DESC
)
SELECT a.visit_id, site_visits.site_url, a.url, a.n
FROM a
INNER JOIN site_visits
ON a.visit_id = site_visits.visit_id
LIMIT 20


In [8]:
%%sql
WITH a AS (
SELECT visit_id,url, COUNT( DISTINCT content_hash ) as unique_content_hashs
FROM http_responses
WHERE content_hash <> ""
GROUP BY visit_id,url
ORDER BY unique_content_hashs DESC
)
SELECT a.visit_id, site_visits.site_url, a.url, a.unique_content_hashs
FROM a
INNER JOIN site_visits
ON a.visit_id = site_visits.visit_id
LIMIT 10


 * sqlite:////home/ndanner_plp/OpenWPM/crawl-data/datadir8000/crawl-data.sqlite
Done.


visit_id,site_url,url,unique_content_hashs
5244274392029786,http://www.titsamateur.com,https://biptolyla.com/auW.ZNyzQu2r9FkyZnT/9U6gbE2C5mlFS/WvQT9RNPDlE/4OMrjrk/0HN/Cc0N0/MLT/g/yNOkTTQK1ZJPnEB-1KcV2-hiaRbO2/5/lGS/WeQs9iNIDeEZ4rMtjTkS0/NVCI0R0UMWT/gwy/OlTQQ/1R,8
8428889483414827,http://www.szjyhh.com.cn,https://www.metamarket.quest/market.js,8
595931919046536,http://www.designfiles.co,https://www.youtube.com/s/player/ac058a09/player_ias.vflset/en_US/base.js,7
6286216596141982,http://www.yoldaolmak.com,https://pagead2.googlesyndication.com/pagead/js/adsbygoogle.js,7
6600483715556861,http://www.xblognetwork.com,https://biptolyla.com/auW.ZNyzQu2r9FkyZnT/9U6gbE2C5mlFS/WvQT9RNPDlE/4OMrjrk/0HN/Cc0N0/MLT/g/yNOkTTQK1ZJPnEB-1KcV2-hiaRbO2/5/lGS/WeQs9iNIDeEZ4rMtjTkS0/NVCI0R0UMWT/gwy/OlTQQ/1R,7
632226293590573,http://www.diabeticlivingonline.com,https://www.googletagservices.com/activeview/js/current/rx_lidar.js?cache=r20110914,6
1354337540513286,http://www.radiopatrulla.com,https://www.youtube.com/s/player/ac058a09/www-embed-player.vflset/www-embed-player.js,6
1887521339126289,http://www.sol8.com,https://analytics.tiktok.com/i18n/pixel/events.js?sdkid=C8138IMVLJR68Q6CP16G&lib=ttq,6
2247437759755395,http://www.laparadadigital.com,https://excellence-prebid.sfo2.cdn.digitaloceanspaces.com/prebid.js,6
3250367981354015,http://www.times24tv.com,https://pagead2.googlesyndication.com/pagead/js/adsbygoogle.js?client=ca-pub-8675921606360886,6


In [6]:
%%sql
SELECT content_hash
FROM http_responses
WHERE visit_id = "5244274392029786" AND url = "https://biptolyla.com/auW.ZNyzQu2r9FkyZnT/9U6gbE2C5mlFS/WvQT9RNPDlE/4OMrjrk/0HN/Cc0N0/MLT/g/yNOkTTQK1ZJPnEB-1KcV2-hiaRbO2/5/lGS/WeQs9iNIDeEZ4rMtjTkS0/NVCI0R0UMWT/gwy/OlTQQ/1R"
GROUP BY content_hash

 * sqlite:////home/ndanner_plp/OpenWPM/crawl-data/datadir8000/crawl-data.sqlite
Done.


content_hash
338359dbbf86b03f293159f6673b33f7f87a0ab0a6930df27bfc69582b824748
339f38f18297db8afa9cdd0f873676e520ab736af47643d682ac158900f78913
5207bf89b594aaa6d83cc89a0aaa8db154216710dbe35af0daab8b826944ae97
6a5c0622bd801957cc56b4dde9dace556dbf84da9ad1011ca8a5616cb4dbbfe4
7b5af75aa61ee6b982d3963610b2a5e3d29f59349c72e37838abb34093dff23e
8e8e82aab5fe939c31aa1ba868332ccd3d6be050ba3fe72c1d67c4172932e171
e33b4d3983818ea9a71dcc41555e08db1d0ac80376f4e5a8f7f34a783d4fb814
e3e9201a9baf494f63fe3166aad405294c7922b56e44510a722d8b0e63b6754c


In [19]:
import difflib
from pprint import pprint


x = difflib.Differ()
str1 = str( db.get(b"338359dbbf86b03f293159f6673b33f7f87a0ab0a6930df27bfc69582b824748"), encoding="utf-8")
str2 = str( db.get(b"339f38f18297db8afa9cdd0f873676e520ab736af47643d682ac158900f78913"), encoding="utf-8")
cmp = x.compare( str1.splitlines(keepends=True), str2.splitlines(keepends=True)  )
for line in filter(lambda s : s[0] == "+" or s[0] == "-", cmp ):
    print(line,end = "\n\n\n\n")


- if(n in o&&r.call(e,o[n],n,o))return!0;return!1})}(),matches:(function(ELEMENT){var matches=ELEMENT.matches||ELEMENT.matchesSelector||ELEMENT.webkitMatchesSelector||ELEMENT.mozMatchesSelector||ELEMENT.msMatchesSelector||ELEMENT.oMatchesSelector;!matches?(ELEMENT.matches=ELEMENT.matchesSelector=function matches(selector){var matches=document.querySelectorAll(selector);var th=this;return Array.prototype.some.call(matches,function(e){return e===th;});}):(ELEMENT.matches=ELEMENT.matchesSelector=matches);})(Element.prototype),closest:(function(ELEMENT){ELEMENT.closest=ELEMENT.closest||function closest(selector){if(!this)return null;if(this.matches(selector))return this;if(!this.parentElement){return null}else return this.parentElement.closest(selector)};})(Element.prototype),}},};try{proto.init(options?options:{});}catch(error){error.situation='banner init';logError(error);}})({"ads":[{"iframeHeight":250,"iframeSrc":"https://biptolyla.com/aIWJZ-y.PL3MBN1Oc_2QhRaSbT2-5VlWSXWYQ_9aNbDcEd4-Mf

In [30]:
%%sql
SELECT *
FROM site_visits
WHERE visit_id = "7805288562003358" OR visit_id = "6723308048261410"

   sqlite:///../../datadirTesting/crawl-data.sqlite
   sqlite:////home/ndanner_plp/OpenWPM/crawl-data/datadir2000/crawl-data.sqlite
 * sqlite:////home/ndanner_plp/OpenWPM/crawl-data/datadir8000/crawl-data.sqlite
Done.


visit_id,browser_id,site_url,site_rank
6723308048261410,4057236297,http://www.revistafama.com,7127
7805288562003358,4057236297,http://www.xiangyunchenbu.com,6724


In [3]:
r = requests.get("http://localhost:8000", params={})

In [7]:
r.text

'{"data":"Hello World!"}'